In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1370229,2021-06-14,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1370230,2021-06-15,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1370231,2021-06-16,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1370232,2021-06-17,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
36287,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
36289,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
36291,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
36293,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
36295,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
840190,2021-06-14,Arkansas,Arkansas,5001,2157,38.00,5000,Arkansas,AR,Arkansas,State,3017804
840192,2021-06-15,Arkansas,Arkansas,5001,2162,38.00,5000,Arkansas,AR,Arkansas,State,3017804
840194,2021-06-16,Arkansas,Arkansas,5001,2166,38.00,5000,Arkansas,AR,Arkansas,State,3017804
840196,2021-06-17,Arkansas,Arkansas,5001,2171,38.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1370229,2021-06-14,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1370230,2021-06-15,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1370231,2021-06-16,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1370232,2021-06-17,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-06-18') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
514,2021-06-18,Snohomish,Washington,39756,595.00,53061,WA,County,822083,4836.01,72.38
1026,2021-06-18,Cook,Illinois,556201,11016.00,17031,IL,County,5150233,10799.53,213.89
1537,2021-06-18,Orange,California,272922,5109.00,6059,CA,County,3175692,8594.10,160.88
2047,2021-06-18,Maricopa,Arizona,555891,10193.00,4013,AZ,County,4485414,12393.30,227.25
2557,2021-06-18,Los Angeles,California,1247883,24444.00,6037,CA,County,10039107,12430.22,243.49
...,...,...,...,...,...,...,...,...,...,...,...
1369361,2021-06-18,Wheeler,Oregon,35,1.00,41069,OR,County,1332,2627.63,75.08
1369610,2021-06-18,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1369828,2021-06-18,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1370042,2021-06-18,Loving,Texas,2,0.00,48301,TX,County,169,1183.43,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
514,2021-06-18,Snohomish,Washington,39756,595.00,53061,WA,County,822083,4836.01,72.38,72.38,4836.01
1026,2021-06-18,Cook,Illinois,556201,11016.00,17031,IL,County,5150233,10799.53,213.89,213.89,10799.53
1537,2021-06-18,Orange,California,272922,5109.00,6059,CA,County,3175692,8594.10,160.88,160.88,8594.10
2047,2021-06-18,Maricopa,Arizona,555891,10193.00,4013,AZ,County,4485414,12393.30,227.25,227.25,12393.30
2557,2021-06-18,Los Angeles,California,1247883,24444.00,6037,CA,County,10039107,12430.22,243.49,243.49,12430.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1369361,2021-06-18,Wheeler,Oregon,35,1.00,41069,OR,County,1332,2627.63,75.08,75.08,2627.63
1369610,2021-06-18,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1369828,2021-06-18,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1370042,2021-06-18,Loving,Texas,2,0.00,48301,TX,County,169,1183.43,0.00,0.00,1183.43


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
878637,2021-06-18,Hale,Alabama,2272,78.00,1065,AL,County,14651,15507.47,532.39,532.39,15507.47,1
956607,2021-06-18,Clarke,Alabama,3530,61.00,1025,AL,County,23622,14943.70,258.23,258.23,14943.70,2
687081,2021-06-18,Lowndes,Alabama,1422,54.00,1085,AL,County,9726,14620.60,555.21,555.21,14620.60,3
609064,2021-06-18,Etowah,Alabama,14175,364.00,1055,AL,County,102268,13860.64,355.93,355.93,13860.64,4
510906,2021-06-18,Franklin,Alabama,4313,82.00,1059,AL,County,31362,13752.31,261.46,261.46,13752.31,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296645,2021-06-18,Platte,Wyoming,672,11.00,56031,WY,County,8393,8006.67,131.06,131.06,8006.67,19
838409,2021-06-18,Converse,Wyoming,1030,18.00,56009,WY,County,13822,7451.89,130.23,130.23,7451.89,20
1042994,2021-06-18,Lincoln,Wyoming,1418,14.00,56023,WY,County,19830,7150.78,70.60,70.60,7150.78,21
1161340,2021-06-18,Niobrara,Wyoming,162,2.00,56027,WY,County,2356,6876.06,84.89,84.89,6876.06,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
687081,2021-06-18,Lowndes,Alabama,1422,54.00,1085,AL,County,9726,14620.60,555.21,555.21,14620.60,3,1
878637,2021-06-18,Hale,Alabama,2272,78.00,1065,AL,County,14651,15507.47,532.39,532.39,15507.47,1,2
607260,2021-06-18,Clay,Alabama,1600,59.00,1027,AL,County,13235,12089.16,445.79,445.79,12089.16,20,3
291771,2021-06-18,Walker,Alabama,7338,280.00,1127,AL,County,63521,11552.09,440.80,440.80,11552.09,28,4
608613,2021-06-18,Dallas,Alabama,3607,158.00,1047,AL,County,37196,9697.28,424.78,424.78,9697.28,55,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1042994,2021-06-18,Lincoln,Wyoming,1418,14.00,56023,WY,County,19830,7150.78,70.60,70.60,7150.78,21,19
739281,2021-06-18,Hot Springs,Wyoming,374,3.00,56017,WY,County,4413,8474.96,67.98,67.98,8474.96,17,20
1016916,2021-06-18,Uinta,Wyoming,2302,13.00,56041,WY,County,20226,11381.39,64.27,64.27,11381.39,5,21
290400,2021-06-18,Teton,Wyoming,3797,11.00,56039,WY,County,23464,16182.24,46.88,46.88,16182.24,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,72.38,4836.01,19,25
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,72.38,4836.01,19,25
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,72.38,4836.01,19,25
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,72.38,4836.01,19,25
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,72.38,4836.01,19,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1367030,2021-06-14,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1367031,2021-06-15,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1367032,2021-06-16,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1367033,2021-06-17,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
874993,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15507.47,2,1,1.00
874994,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15507.47,2,1,0.00
874995,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15507.47,2,1,0.00
874996,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15507.47,2,1,0.00
874997,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,532.39,15507.47,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060518,2021-06-14,Crook,Wyoming,452,12.00,56011,WY,County,7584,5959.92,158.23,158.23,6025.84,7,23,0.00
1060519,2021-06-15,Crook,Wyoming,456,12.00,56011,WY,County,7584,6012.66,158.23,158.23,6025.84,7,23,4.00
1060520,2021-06-16,Crook,Wyoming,456,12.00,56011,WY,County,7584,6012.66,158.23,158.23,6025.84,7,23,0.00
1060521,2021-06-17,Crook,Wyoming,457,12.00,56011,WY,County,7584,6025.84,158.23,158.23,6025.84,7,23,1.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
684329,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14620.60,1,3,1.00,0.00
684330,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14620.60,1,3,0.00,0.00
684331,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14620.60,1,3,0.00,0.00
684332,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14620.60,1,3,0.00,0.00
684333,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14620.60,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
682074,2021-06-14,Albany,Wyoming,4569,15.00,56001,WY,County,38880,11751.54,38.58,38.58,11787.55,23,4,5.00,0.00
682075,2021-06-15,Albany,Wyoming,4573,15.00,56001,WY,County,38880,11761.83,38.58,38.58,11787.55,23,4,4.00,0.00
682076,2021-06-16,Albany,Wyoming,4576,15.00,56001,WY,County,38880,11769.55,38.58,38.58,11787.55,23,4,3.00,0.00
682077,2021-06-17,Albany,Wyoming,4580,15.00,56001,WY,County,38880,11779.84,38.58,38.58,11787.55,23,4,4.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-06-18') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
348407,2021-06-18,Imperial,California,28801,740.00,6025,CA,County,181215,15893.28,408.35,408.35,15893.28,1,2,5.00,0.00
2557,2021-06-18,Los Angeles,California,1247883,24444.00,6037,CA,County,10039107,12430.22,243.49,243.49,12430.22,2,5,184.00,5.00
173291,2021-06-18,San Bernardino,California,299186,4811.00,6071,CA,County,2180085,13723.59,220.68,220.68,13723.59,3,4,35.00,14.00
610820,2021-06-18,Inyo,California,1434,38.00,6027,CA,County,18039,7949.44,210.65,210.65,7949.44,4,25,0.00,0.00
77397,2021-06-18,Stanislaus,California,63387,1069.00,6099,CA,County,550660,11511.10,194.13,194.13,11511.10,5,9,50.00,0.00
61555,2021-06-18,San Joaquin,California,74686,1438.00,6077,CA,County,762148,9799.41,188.68,188.68,9799.41,6,17,57.00,0.00
35817,2021-06-18,Riverside,California,301603,4627.00,6065,CA,County,2470546,12207.95,187.29,187.29,12207.95,7,7,60.00,2.00
95994,2021-06-18,Tulare,California,49814,849.00,6107,CA,County,466195,10685.23,182.11,182.11,10685.23,8,10,15.00,0.00
34879,2021-06-18,Fresno,California,102867,1722.00,6019,CA,County,999101,10295.96,172.35,172.35,10295.96,9,14,25.00,0.00
553433,2021-06-18,Merced,California,32291,478.00,6047,CA,County,277680,11628.85,172.14,172.14,11628.85,10,8,7.00,1.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1296196,2021-06-18,Lassen,California,5784,24.00,6035,CA,County,30573,18918.65,78.50,78.50,18918.65,35,1,0.00,0.00
348407,2021-06-18,Imperial,California,28801,740.00,6025,CA,County,181215,15893.28,408.35,408.35,15893.28,1,2,5.00,0.00
741020,2021-06-18,Kings,California,23189,247.00,6031,CA,County,152940,15162.16,161.50,161.50,15162.16,11,3,4.00,0.00
173291,2021-06-18,San Bernardino,California,299186,4811.00,6071,CA,County,2180085,13723.59,220.68,220.68,13723.59,3,4,35.00,14.00
2557,2021-06-18,Los Angeles,California,1247883,24444.00,6037,CA,County,10039107,12430.22,243.49,243.49,12430.22,2,5,184.00,5.00
217953,2021-06-18,Kern,California,110734,1402.00,6029,CA,County,900202,12301.02,155.74,155.74,12301.02,13,6,24.00,1.00
35817,2021-06-18,Riverside,California,301603,4627.00,6065,CA,County,2470546,12207.95,187.29,187.29,12207.95,7,7,60.00,2.00
553433,2021-06-18,Merced,California,32291,478.00,6047,CA,County,277680,11628.85,172.14,172.14,11628.85,10,8,7.00,1.00
77397,2021-06-18,Stanislaus,California,63387,1069.00,6099,CA,County,550660,11511.10,194.13,194.13,11511.10,5,9,50.00,0.00
95994,2021-06-18,Tulare,California,49814,849.00,6107,CA,County,466195,10685.23,182.11,182.11,10685.23,8,10,15.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
348407,2021-06-18,Imperial,California,28801,740.00,6025,CA,County,181215,15893.28,408.35,408.35,15893.28,1,2,5.00,0.00
2557,2021-06-18,Los Angeles,California,1247883,24444.00,6037,CA,County,10039107,12430.22,243.49,243.49,12430.22,2,5,184.00,5.00
173291,2021-06-18,San Bernardino,California,299186,4811.00,6071,CA,County,2180085,13723.59,220.68,220.68,13723.59,3,4,35.00,14.00
610820,2021-06-18,Inyo,California,1434,38.00,6027,CA,County,18039,7949.44,210.65,210.65,7949.44,4,25,0.00,0.00
77397,2021-06-18,Stanislaus,California,63387,1069.00,6099,CA,County,550660,11511.10,194.13,194.13,11511.10,5,9,50.00,0.00
61555,2021-06-18,San Joaquin,California,74686,1438.00,6077,CA,County,762148,9799.41,188.68,188.68,9799.41,6,17,57.00,0.00
35817,2021-06-18,Riverside,California,301603,4627.00,6065,CA,County,2470546,12207.95,187.29,187.29,12207.95,7,7,60.00,2.00
95994,2021-06-18,Tulare,California,49814,849.00,6107,CA,County,466195,10685.23,182.11,182.11,10685.23,8,10,15.00,0.00
34879,2021-06-18,Fresno,California,102867,1722.00,6019,CA,County,999101,10295.96,172.35,172.35,10295.96,9,14,25.00,0.00
553433,2021-06-18,Merced,California,32291,478.00,6047,CA,County,277680,11628.85,172.14,172.14,11628.85,10,8,7.00,1.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,408.35,15893.28,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,408.35,15893.28,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,408.35,15893.28,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,408.35,15893.28,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,408.35,15893.28,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4572,06/14/21,Lassen,5775,24.00,18889.22,78.50,78.50,18918.65,35,1,0.00,0.00
4573,06/15/21,Lassen,5775,24.00,18889.22,78.50,78.50,18918.65,35,1,0.00,0.00
4574,06/16/21,Lassen,5782,24.00,18912.11,78.50,78.50,18918.65,35,1,7.00,0.00
4575,06/17/21,Lassen,5784,24.00,18918.65,78.50,78.50,18918.65,35,1,2.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,408.35,15893.28,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,408.35,15893.28,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,408.35,15893.28,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,408.35,15893.28,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,408.35,15893.28,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4658,06/14/21,Merced,32272,474.00,11622.01,170.70,172.14,11628.85,10,8,19.00,0.00
4659,06/15/21,Merced,32273,476.00,11622.37,171.42,172.14,11628.85,10,8,1.00,2.00
4660,06/16/21,Merced,32278,477.00,11624.17,171.78,172.14,11628.85,10,8,5.00,1.00
4661,06/17/21,Merced,32284,477.00,11626.33,171.78,172.14,11628.85,10,8,6.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)